本文参考 verl 中的 `multiturn_sft_dataset.py` 代码实现。

通常来说，多轮对话（包含工具调用）的采样轨迹形式如下。

In [4]:
messages = [
    {"role": "user", "content": "Hello, I want to check the weather in Beijing."},
    {"role": "assistant", "content": "Sure, I can help with that. Let me invoke the weather API to get the latest weather information. <tool_call>check_weather(city='Beijing')</tool_call>"},
    {"role": "tool", "content": "2025-12-25, Beijing, sunny, 20C"},
    {"role": "assistant", "content": "The weather in Beijing is sunny with a temperature of 20 degrees Celsius."},
    {"role": "user", "content": "What about Shanghai?"},
    {"role": "assistant", "content": "Sure, I can help with that. Let me invoke the weather API to get the latest weather information. <tool_call>check_weather(city='Shanghai')</tool_call>"},
    {"role": "tool", "content": "2025-12-25, Shanghai, cloudy, 18C"},
    {"role": "assistant", "content": "The weather in Shanghai is cloudy with a temperature of 18 degrees Celsius. If you have any other questions, feel free to ask."},
]

如下对该轨迹进行处理，得到 message_tokens，loss_mask 以及 attention_mask。

In [11]:
from transformers import Qwen2Tokenizer
tokenizer = Qwen2Tokenizer.from_pretrained("/mnt/dolphinfs/hdd_pool/docker/user/hadoop-mtsearch-assistant/ai-search/deepsearch_files/LLMbasemodels/huggingface.co/Qwen/Qwen2.5-32B-Instruct")
tokenizer

Qwen2Tokenizer(name_or_path='/mnt/dolphinfs/hdd_pool/docker/user/hadoop-mtsearch-assistant/ai-search/deepsearch_files/LLMbasemodels/huggingface.co/Qwen/Qwen2.5-32B-Instruct', vocab_size=151643, model_max_length=131072, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalize

对 i = 3 的消息进行处理，此时 role 是 assistant，因此需要梯度。

In [14]:
i = 3

In [17]:
prev_applied_text = tokenizer.apply_chat_template(
    messages[:i], 
    tokenize=False, 
    add_generation_prompt=False,
    enable_thinking=False,
    tools=[{
        "type": "function",
        "function": {
            "name": "check_weather",
            "description": "Check the weather in a specific city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city name"
                    }
                },
                "required": ["city"]
            }
        }
    }]
)

prev_applied_text_w_generation_prompt = tokenizer.apply_chat_template(
    messages[:i], 
    tokenize=False, 
    add_generation_prompt=True,
    enable_thinking=False,
    tools=[{
        "type": "function",
        "function": {
            "name": "check_weather",
            "description": "Check the weather in a specific city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city name"
                    }
                },
                "required": ["city"]
            }
        }
    }]
)

prev_applied_text, prev_applied_text_w_generation_prompt

('<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name": "check_weather", "description": "Check the weather in a specific city.", "parameters": {"type": "object", "properties": {"city": {"type": "string", "description": "The city name"}}, "required": ["city"]}}}\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{{"name": <function-name>, "arguments": <args-json-object>}}\n</tool_call><|im_end|>\n<|im_start|>user\nHello, I want to check the weather in Beijing.<|im_end|>\n<|im_start|>assistant\nSure, I can help with that. Let me invoke the weather API to get the latest weather information. <tool_call>check_weather(city=\'Beijing\')</tool_call><|im_end|>\

In [22]:
cur_applied_text = tokenizer.apply_chat_template(
    messages[:i+1], 
    tokenize=False, 
    add_generation_prompt=False,
    enable_thinking=False,
    tools=[{
        "type": "function",
        "function": {
            "name": "check_weather",
            "description": "Check the weather in a specific city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city name"
                    }
                },
                "required": ["city"]
            }
        }
    }]
)

cur_applied_text

'<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name": "check_weather", "description": "Check the weather in a specific city.", "parameters": {"type": "object", "properties": {"city": {"type": "string", "description": "The city name"}}, "required": ["city"]}}}\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{{"name": <function-name>, "arguments": <args-json-object>}}\n</tool_call><|im_end|>\n<|im_start|>user\nHello, I want to check the weather in Beijing.<|im_end|>\n<|im_start|>assistant\nSure, I can help with that. Let me invoke the weather API to get the latest weather information. <tool_call>check_weather(city=\'Beijing\')</tool_call><|im_end|>\n

In [19]:
generation_prompt_text = prev_applied_text_w_generation_prompt[len(prev_applied_text):]
generation_prompt_text, cur_applied_text[len(prev_applied_text):]

('<|im_start|>assistant\n',
 '<|im_start|>assistant\nThe weather in Beijing is sunny with a temperature of 20 degrees Celsius.<|im_end|>\n')

In [26]:
generation_prompt_tokens = tokenizer.encode(
    generation_prompt_text,
    add_special_tokens=False,
)
_message_tokens = tokenizer.encode(
    cur_applied_text[len(prev_applied_text_w_generation_prompt):],
    add_special_tokens=False,
)
message_tokens = generation_prompt_tokens + _message_tokens
message_tokens, len(message_tokens)

([151644,
  77091,
  198,
  785,
  9104,
  304,
  26549,
  374,
  39698,
  448,
  264,
  9315,
  315,
  220,
  17,
  15,
  12348,
  61347,
  13,
  151645,
  198],
 21)

In [25]:
loss_mask = [0] * (len(generation_prompt_tokens)) + [1] * (
    len(message_tokens) - len(generation_prompt_tokens)
)
loss_mask, len(loss_mask)

([0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 21)

In [27]:
attention_mask = [1] * len(message_tokens)
len(attention_mask)

21